In [1]:
import optuna
import joblib
import datetime

/Users/kmenear/Projects/envSulfurTES/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from stesml.model_tools import build_train_test_model

/Users/kmenear/Projects/envSulfurTES/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
def objective(trial):   
    # Save the study before running the next trial
    joblib.dump(study, "../studies/study_" + model_type + "_" + datetime.datetime.now().strftime("%Y%m%d-%H") + ".pkl")
    
    if model_type == 'NN':
        scale = True
        n_layers = trial.suggest_int("n_layers", 1, 5)
        n_hidden_units = trial.suggest_int("n_hidden_units", 10, 100)
        batch_size = trial.suggest_int("batch_size", 10, 10000, log=True)
        parameters = {'n_layers': n_layers, 'n_hidden_units': n_hidden_units, 'batch_size': batch_size}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters, n_repeats)
    elif model_type == 'XGBoost':
        scale = False
        learning_rate = trial.suggest_float("learning_rate", 0.01, 1)
        subsample = trial.suggest_float("subsample", 0.01, 1)
        colsample_bytree = trial.suggest_float("colsample_bytree", 0.01, 1)
        parameters = {'learning_rate': learning_rate, 'subsample': subsample, 'colsample_bytree': colsample_bytree}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters, n_repeats)
    elif model_type == 'RandomForest':
        scale = False
        n_estimators = trial.suggest_int("n_estimators", 1, 200, log=True)
        parameters = {'n_estimators': n_estimators}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters, n_repeats)
    
    return result

In [4]:
data_dir = "../data/Sulfur_Models/"
model_type = 'XGBoost' # Options: NN, XGBoost, RandomForest
target = 'h' # Options: Tavg, h
metric = 'rmse' # Options: rmse, r2
n_repeats = 2 # Number of times to repeat 5-fold CV. Each repeat gives a different shuffle.

if metric == 'rmse':
    direction = 'minimize'
elif metric == 'r2':
    directon = 'maximize'
    
load_study = False
study_name = "study_NN_20220630-18.pkl"

In [5]:
if load_study:
    study = joblib.load("../studies/" + study_name)
    print("Best trial until now:")
    print(" Value: ", study.best_trial.value)
    print(" Params: ")
    for key, value in study.best_trial.params.items():
        print(f"    {key}: {value}")
else:
    study = optuna.create_study(direction=direction)

[I 2022-07-01 14:34:02,165] A new study created in memory with name: no-name-e66263ef-66ed-4289-bf64-7510619f76a5


In [ ]:
study.optimize(objective, n_trials=50)

[0]	test-rmse:56.49898
[20]	test-rmse:6.20071
[40]	test-rmse:6.17161
[53]	test-rmse:6.16392
Split #0, This Result: 6.1638, Average Result: 6.1638
[0]	test-rmse:55.44422
[20]	test-rmse:5.37929
[40]	test-rmse:4.93975
[60]	test-rmse:4.92467
[78]	test-rmse:4.93980
Split #1, This Result: 4.9396, Average Result: 5.5517
[0]	test-rmse:58.25188
[20]	test-rmse:5.66033
[34]	test-rmse:5.78107
Split #2, This Result: 5.7811, Average Result: 5.6282
[0]	test-rmse:57.85572
[20]	test-rmse:10.05279
[40]	test-rmse:9.92145
[60]	test-rmse:9.91279
[80]	test-rmse:9.91233
[97]	test-rmse:9.91699
Split #3, This Result: 9.9169, Average Result: 6.7004
[0]	test-rmse:53.21708
[20]	test-rmse:10.88809
[40]	test-rmse:10.91212
[43]	test-rmse:10.90930


[I 2022-07-01 14:36:46,058] Trial 0 finished with value: 7.542160192565623 and parameters: {'learning_rate': 0.6804494199247995, 'subsample': 0.9734399419582662, 'colsample_bytree': 0.28476114455835944}. Best is trial 0 with value: 7.542160192565623.


Split #4, This Result: 10.9093, Average Result: 7.5422
[0]	test-rmse:58.94942
[20]	test-rmse:6.59474
[40]	test-rmse:5.82797
[60]	test-rmse:5.73972
[80]	test-rmse:5.79375
[81]	test-rmse:5.79861
Split #0, This Result: 5.7986, Average Result: 5.7986
[0]	test-rmse:57.40437
[20]	test-rmse:5.35707
[40]	test-rmse:5.27004
[60]	test-rmse:5.20973
[80]	test-rmse:5.09213
[100]	test-rmse:5.01782
[117]	test-rmse:5.06500
Split #1, This Result: 5.0650, Average Result: 5.4318
[0]	test-rmse:61.54017
[20]	test-rmse:4.89445
[40]	test-rmse:4.56871
[60]	test-rmse:4.39087
[80]	test-rmse:4.57102
Split #2, This Result: 4.5956, Average Result: 5.1531
[0]	test-rmse:61.39811
[20]	test-rmse:7.56341
[40]	test-rmse:7.34384
[60]	test-rmse:7.24390
[80]	test-rmse:7.05922
[99]	test-rmse:7.09035
Split #3, This Result: 7.0904, Average Result: 5.6374
[0]	test-rmse:56.04591
[20]	test-rmse:7.54780
[40]	test-rmse:7.63061


[I 2022-07-01 14:39:05,837] Trial 1 finished with value: 6.036038303332059 and parameters: {'learning_rate': 0.44310423872680216, 'subsample': 0.053010282387996474, 'colsample_bytree': 0.7092986033183126}. Best is trial 1 with value: 6.036038303332059.


Split #4, This Result: 7.6306, Average Result: 6.0360
[0]	test-rmse:55.30797
[20]	test-rmse:6.67200
[40]	test-rmse:6.04159
[56]	test-rmse:6.04793
Split #0, This Result: 6.0476, Average Result: 6.0476
[0]	test-rmse:53.52809
[20]	test-rmse:6.18314
[40]	test-rmse:5.67218
[60]	test-rmse:5.63664
[80]	test-rmse:5.56035
[100]	test-rmse:5.57907
[104]	test-rmse:5.56005
Split #1, This Result: 5.5602, Average Result: 5.8039
[0]	test-rmse:57.30185
[20]	test-rmse:4.76254
[40]	test-rmse:4.53146
[60]	test-rmse:4.52208
[80]	test-rmse:4.24417
[100]	test-rmse:4.18213
[120]	test-rmse:4.16423
[140]	test-rmse:4.04307
[160]	test-rmse:4.02719
[180]	test-rmse:3.97627
[200]	test-rmse:3.97050
[220]	test-rmse:3.96150
[240]	test-rmse:3.94610
[255]	test-rmse:3.95220
Split #2, This Result: 3.9521, Average Result: 5.1866
[0]	test-rmse:56.16963
[20]	test-rmse:8.27762
[40]	test-rmse:8.09128
[60]	test-rmse:7.98556
[80]	test-rmse:8.02870
Split #3, This Result: 8.0287, Average Result: 5.8972
[0]	test-rmse:51.33277
[20]	t

...

Split #4, This Result: 6.5346, Average Result: 4.9397
[0]	test-rmse:65.03925
[20]	test-rmse:27.04591
[40]	test-rmse:10.01089
[60]	test-rmse:7.67638
[80]	test-rmse:6.44515
[100]	test-rmse:6.27290
[120]	test-rmse:6.15545
[140]	test-rmse:6.11894
[160]	test-rmse:6.10123
[175]	test-rmse:6.10709
Split #0, This Result: 6.1074, Average Result: 6.1074
[0]	test-rmse:63.53163
[20]	test-rmse:32.62347
[40]	test-rmse:17.55791
[60]	test-rmse:7.55298
[80]	test-rmse:5.20395
[100]	test-rmse:4.95430
[120]	test-rmse:4.73967
[140]	test-rmse:4.72947
[148]	test-rmse:4.72776
Split #1, This Result: 4.7291, Average Result: 5.4182
[0]	test-rmse:67.00224
[20]	test-rmse:28.27018
[40]	test-rmse:14.52432
[60]	test-rmse:8.14483
[80]	test-rmse:6.23706
[100]	test-rmse:5.70122
[120]	test-rmse:5.63046
[125]	test-rmse:5.61801
Split #2, This Result: 5.6203, Average Result: 5.4856
[0]	test-rmse:67.25732
[20]	test-rmse:29.93766
[40]	test-rmse:15.82095
[60]	test-rmse:12.47860
[80]	test-rmse:11.65723
[100]	test-rmse:10.95476
[

[I 2022-07-01 17:16:06,995] Trial 25 finished with value: 7.396078952844628 and parameters: {'learning_rate': 0.11754164601174967, 'subsample': 0.8055339902733245, 'colsample_bytree': 0.6365560213861804}. Best is trial 16 with value: 4.826083472746729.


Split #4, This Result: 10.8547, Average Result: 7.3961
[0]	test-rmse:62.92549
[20]	test-rmse:5.57662
[40]	test-rmse:4.58801
[60]	test-rmse:4.46450
[80]	test-rmse:4.36914
[100]	test-rmse:4.26429
[120]	test-rmse:4.21602
[140]	test-rmse:4.17328
[160]	test-rmse:4.16569
[175]	test-rmse:4.17328
Split #0, This Result: 4.1725, Average Result: 4.1725
[0]	test-rmse:61.13791
[20]	test-rmse:5.25445
[40]	test-rmse:4.51764
[56]	test-rmse:4.51599
Split #1, This Result: 4.5172, Average Result: 4.3448
[0]	test-rmse:65.20738
[20]	test-rmse:3.27439
[40]	test-rmse:3.20849
[44]	test-rmse:3.18597
Split #2, This Result: 3.2196, Average Result: 3.9698
[0]	test-rmse:65.29246
[20]	test-rmse:8.47976
[40]	test-rmse:6.34248
[60]	test-rmse:6.16763
[80]	test-rmse:6.11083
[100]	test-rmse:6.08054
[120]	test-rmse:6.05421
[140]	test-rmse:6.03253
[141]	test-rmse:6.03341
Split #3, This Result: 6.0356, Average Result: 4.4862
[0]	test-rmse:59.90763
[20]	test-rmse:9.50121
[40]	test-rmse:6.95994
[60]	test-rmse:6.74782
[80]	te

[I 2022-07-01 17:22:30,183] Trial 26 finished with value: 4.883351108182593 and parameters: {'learning_rate': 0.220214085508386, 'subsample': 0.575574698526335, 'colsample_bytree': 0.7009265095982337}. Best is trial 16 with value: 4.826083472746729.


Split #4, This Result: 6.4718, Average Result: 4.8834
[0]	test-rmse:67.39712
[20]	test-rmse:59.92324
[40]	test-rmse:52.01353
[60]	test-rmse:47.51354
[80]	test-rmse:42.14040
[100]	test-rmse:39.10972
[120]	test-rmse:34.96452
[140]	test-rmse:31.37480
[160]	test-rmse:28.21280
[180]	test-rmse:26.26291
[200]	test-rmse:24.23812
[220]	test-rmse:22.65294
[240]	test-rmse:21.46711
[260]	test-rmse:20.10366
[280]	test-rmse:18.49381
[300]	test-rmse:17.91054
[320]	test-rmse:16.70272
[340]	test-rmse:15.30160
[360]	test-rmse:14.71481
[380]	test-rmse:13.66668
[400]	test-rmse:12.96445
[420]	test-rmse:12.12550
[440]	test-rmse:11.37253
[460]	test-rmse:11.02053
[480]	test-rmse:10.33358
[500]	test-rmse:9.86098
[520]	test-rmse:9.66024
[540]	test-rmse:9.36498
[560]	test-rmse:9.09462
[580]	test-rmse:8.79707
[600]	test-rmse:8.25310
[620]	test-rmse:8.00029
[640]	test-rmse:7.82906
[660]	test-rmse:7.66101
[680]	test-rmse:7.50025
[700]	test-rmse:7.38756
[720]	test-rmse:7.28303
[740]	test-rmse:7.14680
[760]	test-rmse

In [ ]:
best_params = study.best_params

In [18]:
best_params

{'n_layers': 2, 'n_hidden_units': 79, 'batch_size': 339, 'epochs': 7}

In [21]:
best_value = study.best_value

In [22]:
best_value

1.0137617092656621